## Importing the libraries

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

## Importing the dataset

In [36]:
dataset = pd.read_excel('Credit_Card_Score.xlsx')

le = LabelEncoder()

dataset['Payment_of_Min_Amount'] = le.fit_transform(dataset['Payment_of_Min_Amount'])
dataset['Payment_Behaviour'] = le.fit_transform(dataset['Payment_Behaviour'])
dataset['Credit_Score'] = le.fit_transform(dataset['Credit_Score'])
dataset['Credit_Mix'] = le.fit_transform(dataset['Credit_Mix'])

# dataset = dataset.drop('Id', axis=1)


X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values.reshape(-1,1)


# One-hot encode Y
enc = OneHotEncoder(sparse_output=False)

col = dataset.columns.get_loc('Payment_of_Min_Amount')
encoded_col = enc.fit_transform(X[:, col].reshape(-1, 1))

# Replace the original column with encoded one
X = np.delete(X, col, axis=1)       # remove original column
X = np.hstack((X, encoded_col))     # append encoded columns

col = dataset.columns.get_loc('Payment_Behaviour')
encoded_col = enc.fit_transform(X[:, col].reshape(-1, 1))

# Replace the original column with encoded one
X = np.delete(X, col, axis=1)       # remove original column
X = np.hstack((X, encoded_col))     # append encoded columns


col = dataset.columns.get_loc('Credit_Mix')
encoded_col = enc.fit_transform(X[:, col].reshape(-1, 1))

# Replace the original column with encoded one
X = np.delete(X, col, axis=1)       # remove original column
X = np.hstack((X, encoded_col))     # append encoded columns

Y = enc.fit_transform(y)

In [37]:
dataset.head()

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Monthly_Balance,Payment_Behaviour,Age,...,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month,Credit_Mix,Credit_Score
0,3,7,4,26.822620,265,1,80.415295,312.494089,1,23,...,3,4,3,4,1824.843333,11.27,809.98,49.574949,1,0
1,3,7,4,31.944960,265,1,118.280222,284.629162,1,23,...,3,4,3,4,1824.843333,11.27,809.98,49.574949,1,0
2,3,7,4,28.609352,267,1,81.699521,331.209863,1,23,...,3,4,3,4,1824.843333,11.27,809.98,49.574949,1,0
3,5,4,4,31.377862,268,1,199.458074,223.451310,1,23,...,3,4,3,4,1824.843333,11.27,809.98,49.574949,1,0
4,6,4,4,24.797347,269,1,41.420153,341.489231,1,23,...,3,4,3,4,1824.843333,11.27,809.98,49.574949,1,0


## Normalization

In [17]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

## Split Train Test

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [39]:
print(X_train.shape[1])

65


## Activation Functions

In [40]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def d_sigmoid(x):
    s = sigmoid(x)
    return s * (1 - s)

def relu(x):
    return np.maximum(0,x)

def d_relu(x):
    return (x > 0).astype(float)

def softmax(z):
    z_shift = z - np.max(z, axis=1, keepdims=True)  # stability
    exp_z = np.exp(z_shift)
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

## Loss Function

In [41]:
# ----- Loss functions -----

def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

def d_mse(y_true, y_pred):
    return (y_pred - y_true) / y_true.shape[0]

def cross_entropy_loss(y_true, y_pred):
    eps = 1e-12
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))



def binary_cross_entropy(y_true, y_pred, eps=1e-12):
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -np.mean(y_true * np.log(y_pred) + (1-y_true) * np.log(1 - y_pred))

def binary_cross_entropy_grad(y_true, y_pred, eps=1e-12):
    y_pred = np.clip(y_pred, eps, 1-eps)
    return (-(y_true/y_pred) + (1-y_true) / (1-y_pred)) / y_true.shape[0]

## MLP Class

In [42]:
class MLP:
    def __init__(self, layer_sizes, task='classification', lr=0.001):
        """
        layer_sizes: list, e.g., [n_inputs_dime, hidden1, hidden2, ..., n_outputs_dim]
        task: 'classification' | 'multiclass' | 'regression'
        """
        
        self.lr = lr
        self.layer_sizes = layer_sizes
        self.task = task

        self.W = []
        self.B = []

        for i in range(len(layer_sizes)-1):
            self.W.append(np.random.randn(layer_sizes[i], layer_sizes[i+1]) * 0.01)
            self.B.append(np.zeros((1, layer_sizes[i+1])))

    # ------------Forward Pass-------------
    def forward(self, X):
        self.A = [X]
        self.Z = []

        for i in range(len(self.W)):
            z = self.A[-1] @ self.W[i] + self.B[i]
            self.Z.append(z)

            #Use ReLu for hidden layers
            if i < len(self.W) - 1:
                self.A.append(relu(z))
            else:
                if self.task == 'classification':
                    self.A.append(sigmoid(z))
                elif self.task == 'multiclass':
                    self.A.append(softmax(z))
                else:
                    self.A.append(z) # Regression
        return self.A[-1] 


    def backward(self, y_true):

        m = y_true.shape[0]
        grads_W = [None] * len(self.W)
        grads_B = [None] * len(self.B)

        # Output layer gradient
        if self.task == 'classification':
            dA = (self.A[-1] - y_true) # binary cross entropy simplified
        elif self.task == 'multiclass':
            dA = (self.A[-1] - y_true) # softmax + CE gradient
        else:
            dA = d_mse(y_true, self.A[-1])

        # Backpropagation
        for i in reversed(range(len(self.W))):
            if i == len(self.W) - 1: # output layer
                dZ = dA
            else:
                dZ = dA * d_relu(self.Z[i])

            grads_W[i] = self.A[i].T @ dZ
            grads_B[i] = np.sum(dZ, axis=0, keepdims=True)

            dA = dZ @ self.W[i].T
        # Gradient descent update
        for i in range(len(self.W)):
            self.W[i] -= self.lr * grads_W[i]
            self.B[i] -= self.lr * grads_B[i]
        

     # ---------- Training ----------
    def fit(self, X, y, epochs=500, verbose=True):
        losses = []
        
        for epoch in range(1, epochs+1):
            y_pred = self.forward(X)
            
            if self.task in ["classification", "multiclass"]:
                loss = cross_entropy_loss(y, y_pred)
            else:
                loss = mse_loss(y, y_pred)
                
            self.backward(y)
            losses.append(loss)
            
            if verbose and epoch % 100 == 0:
                print(f"Epoch {epoch} - Loss: {loss:.4f}")
        return losses

    # ---------- Prediction ----------
    def predict(self, X):
        out = self.forward(X)
        
        if self.task == "classification":
            return (out > 0.5).astype(int)
        
        if self.task == "multiclass":
            return np.argmax(out, axis=1)
        
        return out  # regression outputs

## Train and Predict Model on the Test set (Multi Classification)

In [43]:
mlp = MLP([X_train.shape[1],128,64, 32, 16,8, 3], task='multiclass', lr=0.001)
mlp.fit(X_train,y_train, epochs=1000)


y_pred = mlp.predict(X_test)
y_true = np.argmax(y_test, axis=1)
accuracy = np.mean(y_pred == y_true)
print("Accuracy:", accuracy)
y_test_label = np.argmax(y_test, axis=1)
print(y_test_label)
print(y_pred)

Epoch 100 - Loss: 2.3535
Epoch 200 - Loss: 2.3535
Epoch 300 - Loss: 2.3535
Epoch 400 - Loss: 2.3535
Epoch 500 - Loss: 2.3535
Epoch 600 - Loss: 2.3535
Epoch 700 - Loss: 2.3535
Epoch 800 - Loss: 2.3535
Epoch 900 - Loss: 2.3535
Epoch 1000 - Loss: 2.3535
Accuracy: 0.195
[2 2 0 ... 2 2 0]
[0 0 0 ... 0 0 0]


## Confusion Matrix

In [133]:
from sklearn.metrics import confusion_matrix, accuracy_score

confusion_matrix(y_test_label, y_pred)

array([[10,  0,  0],
       [ 0,  9,  0],
       [ 0,  0, 11]])

In [134]:
accuracy_score(y_test_label,y_pred)

1.0